In [32]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import finsymbols
from pandas_datareader import data as pdr

In [3]:
yf.pdr_override() #Work around for yahoo finance
stock = input("Enter a stock ticker symbol: ")

start_year = 2020
start_month = 3
start_day = 6

start = dt.datetime(start_year, start_month, start_day)

now = dt.datetime.now()
df = pdr.get_data_yahoo(stock, start, now)
print(df)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-03-06   70.500000   72.705002   70.307503   72.257500   70.957054   
2020-03-09   65.937500   69.522499   65.750000   66.542503   65.344917   
2020-03-10   69.285004   71.610001   67.342499   71.334999   70.051155   
2020-03-11   69.347504   70.305000   67.964996   68.857498   67.618248   
2020-03-12   63.985001   67.500000   62.000000   62.057499   60.940628   
...                ...         ...         ...         ...         ...   
2023-01-30  144.960007  145.550003  142.850006  143.000000  143.000000   
2023-01-31  142.699997  144.339996  142.279999  144.289993  144.289993   
2023-02-01  143.970001  146.610001  141.320007  145.429993  145.429993   
2023-02-02  148.899994  151.179993  148.169998  150.820007  150.820007   
2023-02-03  148.029999  157.380005  147.830

In [4]:
def MACD(DF, a, b, c):
    """
    Calculate MACD of given DataFrama
    :param DF: Inputed stock data as DataFrame
    :param a: Typically 12 Fast moving average
    :param b: Typically 26 Slow moving average
    :param c: Typlically 9 Moving average of the MACD line
    :return: DF with MACD
    """
    df = DF.copy()
    df["MA_FAST"] = df["Adj Close"].rolling(a).mean()
    df["MA_SLOW"] = df["Adj Close"].rolling(b).mean()
    df["MACD"] = df["MA_FAST"]-df["MA_SLOW"]
    df["SIGNAL"] = df["MACD"].rolling(c).mean()
    df.dropna(inplace = True)
    return df
print(MACD(df, 12, 26, 9))

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-04-23   68.967499   70.437500   68.717499   68.757500   67.520042   
2020-04-24   69.300003   70.752502   69.250000   70.742500   69.469322   
2020-04-27   70.449997   71.135002   69.987503   70.792503   69.518425   
2020-04-28   71.269997   71.457497   69.550003   69.644997   68.391563   
2020-04-29   71.182503   72.417503   70.972504   71.932503   70.637894   
...                ...         ...         ...         ...         ...   
2023-01-30  144.960007  145.550003  142.850006  143.000000  143.000000   
2023-01-31  142.699997  144.339996  142.279999  144.289993  144.289993   
2023-02-01  143.970001  146.610001  141.320007  145.429993  145.429993   
2023-02-02  148.899994  151.179993  148.169998  150.820007  150.820007   
2023-02-03  148.029999  157.380005  147.830002  154.500000  154.500000   

               Volume     MA_FAST    

In [ ]:
def EMA_MAC(DF, a, b, c):
    """
    Calculate EMA of given DataFrama
    :param DF: Inputed stock data as DataFrame
    :param a: Typically 12 Fast moving average
    :param b: Typically 26 Slow moving average
    :param c: Typlically 9 Moving average of the MACD line
    :return: DF with MACD
    """
    df = DF.copy()
    df["MA_FAST"] = df["Adj Close"].ewm(span = a, min_periods = a).mean()
    df["MA_SLOW"] = df["Adj Close"].ewm(span = b, min_periods = b).mean()
    df["MACD"] = df["MA_FAST"]-df["MA_SLOW"]
    df["SIGNAL"] = df["MACD"].ewm(span = c, min_periods = c).mean()
    df.dropna(inplace = True)
    return df

Support and Resistance Levels | Pivot Points
candle <= 15 min : Prev day’s high, low, close 15 min < candle <= 1 hr : Prev Week’s high, low, close 1 hr < candle <= 1 day : Prev Month’s high, low, close

In [ ]:
def s_r_levels(df_day):
    """
    :param df_day: DF in given time frame.
    :return pivot, r1, r2, r3, s1, s2, s3 of day:
    """
    high = round(df_day["High"][-1], 2)
    low = round(df_day["Low"][-1], 2)
    close = round(df_day["Adj Close"][-1], 2)
    pivot = round((high+low+close)/3,2)
    r1 = round((2*pivot - low),2)
    r2 = round((pivot + (high - low)),2)
    r3 = round((high + 2*(pivot - low)),2)
    s1 = round((2*pivot - high),2)
    s2 = round((pivot - (high - low)),2)
    s3 = round((low - 2*(high - pivot)),2)
    return pivot, r1, r2, r3, s1, s2, s3

In [ ]:
def doji(df):
    """
    :param df: Dataframe of stock
    :return: df with doji candle indicator
    """
    doji_df = df.copy()
    avg_size = abs(doji_df["Adj Close"] - doji_df["open"]).median()
    doji_df["doji"] = abs(doji_df["Adj Close"] - doji_df["open"]) <= (0.05 * avg_size)
    return doji_df

In [ ]:
def hammer(DF):
    """
    :param DF: ataframe of stock
    :return: df with hammer candle indicator
    """
    df = DF.copy
    df["hammer"] = (((df["High"] - df["Low"])>3*(df["Open"] - df["Adj Close"])) & \
                   ((df["Adj Close"] - df["Low"])/(.001 + df["High"] - df["Low"]) > 0.6) & \
                   ((df["Open"] - df["Low"])/(.001 + df["High"] - df["Low"]) > 0.6)) & \
                   (abs(df["Adj Close"] - df["Open"]) > 0.1* (df["High"] - df["Low"]))
    return df

In [ ]:
def star(DF):
    """
    :param DF: dataframe of stock
    :return: df with start candle indicator
    """
    df = DF.copy()
    df["sstar"] = (((df["High"] - df["Low"])>3*(df["Open"] - df["Adj Close"])) & \
                   ((df["High"] - df["Adj Close"])/(.001 + df["High"] - df["Low"]) > 0.6) & \
                   ((df["High"] - df["Open"])/(.001 + df["High"] - df["Low"]) > 0.6)) & \
                   (abs(df["Adj Close"] - df["Open"]) > 0.1* (df["High"] - df["Low"]))
    return df

In [ ]:
def maru_bozu(DF):
    """
    :param DF: dataframe of stock
    :return: df with maru_bozu candle indicator
    """
    df = DF.copy()
    avg_candle_size = abs(df["Adj Close"] - df["Open"]).median()
    df["h-c"] = df["High"]-df["Close"]
    df["l-o"] = df["Low"]-df["Open"]
    df["h-o"] = df["High"]-df["Open"]
    df["l-c"] = df["Low"]-df["Adj Close"]
    df["maru_bozu"] = np.where((df["Adj Close"] - df["Open"] > 2*avg_candle_size) & \
                               (df[["h-c","l-o"]].max(axis=1) < 0.005*avg_candle_size),"maru_bozu_green",
                               np.where((df["Open"] - df["Adj Close"] > 2*avg_candle_size) & \
                               (abs(df[["h-o","l-c"]]).max(axis=1) < 0.005*avg_candle_size),"maru_bozu_red",False))
    df.drop(["h-c","l-o","h-o","l-c"],axis=1,inplace=True)
    return df

In [45]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

stocks =  df['Symbol']
for i in range(len(stocks)):
    stock = stocks[i]
    if '.' in stock:
        stock = stock.replace('.', '-')
        print(stock)
    start_year = 2023
    start_month = 2
    start_day = 1
    start = dt.datetime(start_year, start_month, start_day)
    now = dt.datetime.now()
    df = pdr.get_data_yahoo(stock, start, now)

BRK-B
BF-B
